# Exploring BFS

First, include some libraries

In [ ]:
# Library imports

#
# Import tensor class
#
from fibertree import Payload, Fiber, Tensor, TensorImage

#
# Import display classes/utilities
#
from IPython.display import display # to display images

def displayTensor(t):
    display(TensorImage(t).im)

#
# Helper for data directory
#
import os

data_dir = "../data"

def datafileName(filename):
    return os.path.join(data_dir, filename)



## Graph Inputs


In [ ]:
# Adjacency matrix
a = Tensor.fromUncompressed([ "S", "D"],
                            [ [ 0, 1, 1, 0, 0, 0 ],
                              [ 0, 0, 1, 1, 0, 0 ],
                              [ 0, 0, 0, 1, 1, 0 ],
                              [ 0, 0, 0, 0, 1, 1 ],
                              [ 1, 0, 0, 0, 0, 1 ],
                              [ 1, 1, 0, 0, 0, 0 ] ])

# Fringe (current and next)
f0 = Tensor.fromUncompressed([ "D" ], [ 1, 0, 0, 0, 0, 0 ])

# Distance
d = Tensor(rank_ids=[ "S" ])

print("Adjanceny Matrix")
displayTensor(a)

print("Distance Vector")
displayTensor(d)

print("CUrrent Fringe")
displayTensor(f0)





## Naive BFS

This version traverses all neighbors of each source node, even if there already is a distance. So there is a check to not create a new distance.

In [ ]:
# Get root fibers
a_s = a.root()
f0_d = f0.root()
d_d = d.root()

level = 1


while (f0_d.countValues() > 0):
    print("\n\n")
    print(f"Level {level} fringe")
    displayTensor(f0_d.nonEmpty())
    print(f"Level {level} distances")
    displayTensor(d_d)   

    # Create a new next fringe (f1)
    
    f1 = Tensor(rank_ids=[ "D" ]) 
    f1_d = f1.root()

    for s, (_, a_d) in f0_d & a_s:
        print(f"Processing source {s}")
        print(f"Neighbors:\n {a_d}")

        for d, (f1_ref, (d_ref, _)) in f1_d << (d_d << a_d):
            print(f"  Processing destination {d} = {d_ref}")

            if Payload.isEmpty(d_ref):
                print(f"  Adding destination {d}")

                f1_ref += 1
                d_ref += level

    # Move to next level
    
    level += 1
    
    # Copy next fringe to current fringe
    
    f0 = f1
    f0_d = f0.root()


print("Final Distances")
displayTensor(d_d)

## Optimized BFS

Avoid processing of any destination node that already has a distance by subtracting the distance array from the neighbors

## Graph Inputs


In [ ]:
# Adjacency matrix
a = Tensor.fromUncompressed([ "S", "D"],
                            [ [ 0, 1, 1, 0, 0, 0 ],
                              [ 0, 0, 1, 1, 0, 0 ],
                              [ 0, 0, 0, 1, 1, 0 ],
                              [ 0, 0, 0, 0, 1, 1 ],
                              [ 1, 0, 0, 0, 0, 1 ],
                              [ 1, 1, 0, 0, 0, 0 ] ])

# Fringe (current and next)
f0 = Tensor.fromUncompressed([ "D" ], [ 1, 0, 0, 0, 0, 0 ])

# Distance
d = Tensor(rank_ids=[ "S" ])

print("Adjanceny Matrix")
displayTensor(a)

print("Distance Vector")
displayTensor(d)

print("CUrrent Fringe")
displayTensor(f0)





In [ ]:
# Get root fibers
a_s = a.root()
f0_d = f0.root()
d_d = d.root()

level = 1


while (f0_d.countValues() > 0):
    print("\n\n")
    print(f"Level {level} fringe")
    displayTensor(f0_d.nonEmpty())
    print(f"Level {level} distances")
    displayTensor(d_d)   

    # Create a new next fringe (f1)
    
    f1 = Tensor(rank_ids=[ "D" ]) 
    f1_d = f1.root()

    for s, (_, a_d) in f0_d & a_s:
        print(f"Processing source {s}")
        print(f"Neighbors:\n {a_d}")

        for d, (f1_ref, (d_ref, _)) in f1_d << (d_d << (a_d - d_d)):
            print(f"  Processing destination {d} = {d_ref}")
            print(f"  Adding destination {d}")

            f1_ref += 1
            d_ref += level

    # Move to next level
    
    level += 1
    
    # Copy next fringe to current fringe
    
    f0 = f1
    f0_d = f0.root()

print("Final Distances")
displayTensor(d_d)

## Testing area

For running alternative algorithms